# MAX + TF.js: Review Text Generator

https://github.com/IBM/MAX-Review-Text-Generator

notes/links/etc

- `Netron`: https://github.com/lutzroeder/Netron
- `TF.js Converter`: https://github.com/tensorflow/tfjs-converter
- `TF.js API`: https://js.tensorflow.org/api/latest


<br>

### Table of Contents

1. [Prerequisites](#Prerequisites)
1. [Run inference using the model](#Run-inference-using-the-model)
1. [Convert the model to a web-friendly format](#Convert-the-model-to-a-web-friendly-format)


<br>

## Prerequisites

- Clone the MAX Review Text Generator GitHub repository:

    ```
    git clone https://github.com/IBM/MAX-Review-Text-Generator.git
    ```

- Download the model artifacts for the Review Text Generator:

    [http://max-assets.s3-api.us-geo.objectstorage.softlayer.net/keras/generative_lang_model/generative_lang_model.h5](http://max-assets.s3-api.us-geo.objectstorage.softlayer.net/keras/generative_lang_model/generative_lang_model.h5)

    For example, from a terminal window:
    
    ```
    curl -O http://max-assets.s3-api.us-geo.objectstorage.softlayer.net/keras/generative_lang_model/generative_lang_model.h5
    ```

<br>


In [ ]:
# This notebook has been tested with Python version 3.6.6
!python --version


In [ ]:
# This notebook has been tested with tensorflow 1.12.0, tensorflowjs 0.8.0, and numpy 1.15.1
!pip show tensorflow tensorflowjs numpy


In [ ]:
# Uncomment to install the packages needed

# !pip install -Iv tensorflow
# !pip install -Iv tensorflowjs
# !pip install -Iv numpy

# Restart the kernel after installation completes.


<br>

<strong>NOTE</strong>: Update the variables with the appropriate directory path


In [ ]:
# full path to cloned repo
review_text_generator = '/Users/va/Desktop/max/repos/MAX-Review-Text-Generator'

# full path to extracted frozen graph
keras_model_path = '/Users/va/Desktop/max/review-text-generator/model/generative_lang_model.h5'


<br>
<hr>

# Run inference using the model


In [ ]:
import tensorflow as tf

print('TF versions:', tf.GIT_VERSION, tf.VERSION)


<br>

Load Keras model


In [ ]:
# load the keras model
def load_keras_model(path):
    model = tf.keras.models.load_model(
        path,
        custom_objects=None,
        compile=True
    )
    return model


In [ ]:
model = load_keras_model(keras_model_path)

<br>

Load assets


In [ ]:
import json

# https://github.com/IBM/MAX-Review-Text-Generator/blob/master/core/backend.py#L28
def load_assets(path):
    p1 = '{}/char_indices.txt'.format(path)
    print(p1)
    with open(p1) as f:
        char_indices = json.loads(f.read())
        chars = sorted(char_indices.keys())
        num_chars = len(chars)
        
    p2 = '{}/indices_char.txt'.format(path)
    print(p2)
    with open(p2) as f:
        indices_char = json.loads(f.read())

    return char_indices, chars, num_chars, indices_char


In [ ]:
# path to the assets directory in the github repository
model_assets = review_text_generator + '/assets'

char_indices, chars, num_chars, indices_char = load_assets(model_assets)


<br>

Run prediction on sample text


In [ ]:
from timeit import default_timer as timer
import numpy as np


# length required for the seed text. 
# seed text will be padded or truncated as needed
SEED_TEXT_LEN = 256


# https://github.com/IBM/MAX-Review-Text-Generator/blob/master/core/backend.py#L45
def predict(sentence, gen_chars=50):
    # Generate text based on seed text.
    sentence = sentence.lower()
    for t, char in enumerate(sentence):
        if char not in char_indices:
            print("Bad char {} at position {}".format(char, t))
            raise ValueError(
                    "Unexpected character '{}' at position {}. "
                    "Only lowercase ASCII characters, spaces, "
                    "and basic punctuation are supported.".format(char, t))

    # The text passed into the model must be exactly SEED_TEXT_LEN
    # characters long, or the model will crash. Pad or truncate.
    if len(sentence) > SEED_TEXT_LEN:
        sentence = sentence[:SEED_TEXT_LEN]
    else:
        sentence = sentence.rjust(SEED_TEXT_LEN)

    generated = ''
    start = timer()
    
    for i in range(gen_chars):
        x = np.zeros((1, SEED_TEXT_LEN, num_chars))

        for t, char in enumerate(sentence):
            x[0, t, char_indices[char]] = 1.

        preds = model.predict(x, verbose=0)[0]

        next_index = _sample(preds)
        next_char = indices_char[str(next_index)]

        generated += next_char
        sentence = sentence[1:] + next_char
    
    end = timer()
    print('predict: {}'.format(end - start))

    return generated


# https://github.com/IBM/MAX-Review-Text-Generator/blob/master/core/backend.py#L36
def _sample(preds, temperature=.6):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


<br>

<strong>NOTE</strong>: Update the variables with the desired text to test

In [ ]:
# sentence = 'Came here last Friday with my friends. Got there around 8:30 and got seated right away. Parking'
sentence = 'heart be still i loved this place. way better than i expected. i had the spicy noodles and they were delicious, flavor great and quality was on point. for desert the sticky rice with mango, i dream about it now. highly recommend if you are in the mood for '


In [ ]:
# run prediction
# this may take a while
generated = predict(sentence)


In [ ]:
# print orginial sentence plus generated text
print (sentence + generated)


<br>
<hr>

# Convert the model to a web-friendly format

[https://github.com/tensorflow/tfjs-converter](https://github.com/tensorflow/tfjs-converter)


```
tensorflowjs_converter \
    --input_format=keras \
    /path/to/keras/model.h5 \
    /path/to/web_asset_output_dir
```


In [ ]:
# check for tensorflowjs_converter
!tensorflowjs_converter --version


<br>

<strong>NOTE</strong>: Update the variables with the appropriate path to save the converted model assets

In [ ]:
# set appropriate desired output path for web format
web_asset_dir = '/Users/va/Desktop/max/review-text-generator/model-tfjs'


In [ ]:
import pathlib

# create directory if it does not exist
pathlib.Path(web_asset_dir).mkdir(parents=True, exist_ok=True)


<br>

Run the converter


In [ ]:

!tensorflowjs_converter \
    --input_format=keras \
    {keras_model_path} \
    {web_asset_dir}


<br>

Print the converted model assets


In [ ]:
import os
import time

print("Web asset directory {}:".format(web_asset_dir))

web_assets = os.listdir(web_asset_dir)
web_assets.sort()

for file in web_assets:
    file_stat = os.stat("{}/{}".format(web_asset_dir,file))
    print(" {} {} {:>20}".format(file.ljust(30), time.ctime(file_stat.st_mtime), file_stat.st_size))
